# **Tutorial 2: Filtering and Common Transformations**

In this part, we will learn how to implement basic 'cleaning' and spatio-temporal transformations of the data which are common in an analysis pipeline. We will also cover a few "gotcha"s to keep an eye out for. Furthermore, we show some good practices for 'cleaning' the data from unreliable users and/or pings. 

## Introduction

Researchers often filter geospatial datasets to focus on users or trajectories that meet specific criteria—such as sufficient activity, coverage within a particular timeframe, or presence within defined geographic areas. For example, a study might analyze “users with at least 14 days of activity in January who visited Central Park.” Beyond identifying qualifying users, researchers often need to retrieve these users’ complete trajectories across a wider region to study questions like “Where else do Central Park visitors travel from or to around the world?”

Obviously, this reduces the scope of the analysis and makes the sample small and oftentimes manageable without the need for a cluster. However, to get that sample it might be necessary to implement some filtering at read time. 

Building such filters typically involves handling polygon operations, coordinate system transformations, and reconciling inconsistent formats from various data sources.

Below we show a two step process:

1.	Select users that meet specified geographical and temporal conditions.  
2.	Extract the complete trajectories from those users

## Load Data

For this example, we will use a third dataset different than the one used in part 1, which has more users and, occasionally, denser signals. 

In [ ]:
import nomad.io.base as loader

filepath_root = 'gc_data_long/'
tc = {
    "user_id": "gc_identifier",
    "timestamp": "unix_ts",
    "x": "dev_x",
    "y": "dev_y",
    "ha":"ha",
    "date":"date"}

traj = loader.from_file(filepath_root, format='parquet', traj_cols=tc)
traj.head()

## Ping Frequency

In [ ]:
# We can investigate the frequency of these GPS signals, which can be very sparse
import matplotlib.pyplot as plt

total_pings = traj.groupby(tc['user_id']).size()
total_hours = (traj[tc['timestamp']].max() - traj[tc['timestamp']].min())//3600

fig, (ax1, ax2) = plt.subplots(figsize=(10,3), ncols=2, sharey=True)

total_pings.hist(ax=ax1, bins= [500*b for b in range(17)])
ax1.set_xlabel('total pings')

# x-axis restricted to bins
(total_pings/total_hours).hist(ax=ax2, bins= [0.25*b for b in range(40)])
ax2.set_xlabel('ping frequency (records/hour)')

plt.show()

We note **heterogeneity in the ping frequencies** and the volume of data. However, a common gotcha in this type of analysis is to assume that *more records* = *more complete signals*. Due to how the data is collected---often in short bursts---there are users with fewer pings but **more evenly distributed** over the study period. Furthermore, many pings might have such high levels of error, that they shouldn't even be considered in the analysis. Some minimal cleaning already changes the picture. 

In [ ]:
from nomad import filters

# Remove horizontal accuracy over 50m
traj_ds = traj.loc[traj[tc['ha']] < 50]
print(f"{((traj[tc['ha']]>50).mean()*100).round(3)}% pings have 'ha' > 50m")
# Downsampling pings to once per minute
num_mins=1 # Try 5
traj_ds = filters.downsample(traj_ds, periods=num_mins, verbose=True, traj_cols=tc)

In [ ]:
total_pings_filtered = traj_ds.groupby(tc['user_id']).size()

fig, (ax1, ax2) = plt.subplots(figsize=(10,3), ncols=2, sharey=True)

total_pings.hist(ax=ax1, bins= [500*b for b in range(17)], alpha=0.5, label='raw')
total_pings_filtered.hist(ax=ax1, bins= [500*b for b in range(17)], color='darkorange', alpha=0.5, label='filtered')
ax1.legend()
ax1.set_xlabel('total pings')

(total_pings/total_hours).hist(ax=ax2, bins= [0.25*b for b in range(30)], alpha=0.5, label='raw')
(total_pings_filtered/total_hours).hist(ax=ax2, bins= [0.25*b for b in range(30)], color='darkorange', alpha=0.5, label='filtered')
ax2.set_xlabel('ping frequency (records/hour)')
ax2.legend()

plt.show()

`nomad`'s `filters.downsample` method automatically takes care of grouping by user if the name of a user id column is passed (in this case via `traj_cols`), but it would also work for a single trajectory or on a `pandas.datetime64` datetime column if available.

In [ ]:
import pandas as pd
import numpy as np

from shapely.geometry import Polygon
from datetime import datetime
import matplotlib.patches as patches

from nomad.filters import to_projection, filter_users, q_filter

## Spatial filters and Projections

Suppose we are interested in an analysis of users who visited a park during multiple days, e.g. **3 days**, of January 2024. In particular, we might want to know which other locations these users tend to visit to try and get some insights. Naturally, we do this through a spatial join of each user's coordinates with the polygon that represents the park. Here are some possible immediate challenges that could make an analysis `ad-hoc`

- The polygon could be in **spherical coordinates** (also called **geographic coordinates**) and the data in **projected coordinates**. For example, in `(longitude, latitude)` coordinates with `crs='EPSG:4326'`, while the data has ``` (x, y)``` coordinates from the Web Mercator crs (`'EPSG:3857'`).

- The polygon could be represented as a Well-Known-Text string (`wkt`), `shapely` polygon (or multipolygon), or a `Geopandas.GeoSeries`

We can get a `shapely.Polygon` representing the park querying from the file `garden_city.geojson` but, for the sake of completeness, we will assume that the polygon is given by a `wkt`. `geopandas` and `shapely` already wrap easy to use methods for spatial joins.

Add an image of web mercator crs vs geographical coordinates crs

### Load the geometry data

In [ ]:
import geopandas as gpd
import pyproj # for coordinate reprojection
from shapely.geometry import Polygon, box

city = gpd.read_file("garden_city.geojson").set_index('building_id')
outer_box = box(*city.total_bounds).buffer(0.00015, join_style='mitre') # "background" box
# poly = city.geometry.loc['p-x13-y11'] #uncomment for comparing

poly_wkt = 'POLYGON ((-38.31767437672955 36.66929877196072, -38.31767437672955 36.66973109167419,'\
                     '-38.31821336590002 36.66973109167419, -38.31821336590002 36.66929877196072,'\
                     '-38.31767437672955 36.66929877196072))'
poly = gpd.GeoSeries.from_wkt([poly_wkt])[0]

### Check if a user has pings inside the park 

We start by analyzing the data from a single user. Checking which pings fall inside a given polygon is trivial with `geopandas.GeoDataFrame.within`. However, the polygon and the passed pings must have the same CRS. Reprojection is also quite simple if we cast our dataset as a `GeoDataFrame`. 

In [ ]:
user_df = traj.loc[(traj[tc['user_id']] == 'kind_perlman')]

# Cast to GeoDataFrame and reproject (easy)
user_df = gpd.GeoDataFrame(user_df,
                           geometry=gpd.points_from_xy(user_df['dev_x'], user_df['dev_y']),
                           crs='EPSG:3857').to_crs('EPSG:4326')

user_df['in_park'] = user_df.within(poly)
user_df = gpd.clip(user_df, outer_box) # For plotting purposes

In [ ]:
import shapely.plotting as shp_plt
# for pretty plot
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_axis_off()

shp_plt.plot_polygon(outer_box, ax=ax, add_points=False, color='#0e0e0e')
city.plot(ax=ax, edgecolor='white', color='#2c353c')
shp_plt.plot_polygon(poly, ax=ax, add_points=False, color='#0fad5a', alpha=0.5)
user_df.plot(ax=ax, column='in_park', marker='o', markersize=12, alpha=(0.4+user_df.in_park*0.6), cmap='Reds')
ax.set_title("Pings inside 'p-x13-y11' (single user)")

plt.show()

### Qualification constraints: minimum active days

Trajectories are **noisy**, so some of the pings in the park might actually come from the street. Thus, to include our user in the final analysis we want to make sure they have a sufficient number of pings inside the park in each of, at least, 3 days. This could remove any **spurious 'visits'** which are *actually noise*.

In [ ]:
user_df.loc[user_df['in_park']].groupby('date').size()

We again face the issue of pings possibly being concentrated in short bursts, in which case, *even 60 pings could happen in the span of a few minutes*. Properly detecting periods of time in which individuals stopped will be explained in the next notebook. For now, we can deal with spurious visits by using `filters.downsample` again. Our criterion can be that **in at least 3 days there are at least 3 different pings in the park separated by at least 3 minutes** (feel free to change this at your leisure)

In [ ]:
park_pings_ds = filters.downsample(user_df.loc[user_df['in_park']], periods=3, traj_cols=tc)
park_pings_ds.groupby('date')['in_park'].size()

Excellent. We see that our user makes the cut and can be incorporated in our sample. 

### Qualification constraints: signal completeness (q)

As with any dataset, if the data for a user is highly incomplete, then results might not be reliable due to *incompleteness bias*. Imagine trying to analyze whether visitors to the park tend to come back the next day, but one of your 'visitors' only has data every 5 days. We quantify the *completeness* of a user with a statistic $q$ (or, conversely its *sparsity* would be $1-q$) that, given a temporal resolution (typically an hour, or a day) provides the percentage of 'time buckets' with any data at that resolution.



In [ ]:
filters.completeness(user_df, periods=1, freq='h', traj_cols=tc)

In [ ]:
user_df.unix_ts.is_monotonic_increasing

## Coordinate system projection

The first step after loading the dataset is to ensure the dataset is in the expected coordinate system.

Many geospatial datasets come in spherical coordiantes latitude/longitude (EPSG:4326). However, spatial analyses---like joins of points in polygons, computing buffers, or clustering pings---might benefit from computing euclidean distances. Thus projected planar coordinates (like EPSG:3857) are commonly used. Nomad's `to_projection` method creates new columns `x` and `y` with projected coordinates in any coordinate reference system (CRS) recognized by PyProj. 

In [ ]:
# Project to EPSG:3857 (Web Mercator)
projected_x, projected_y = to_projection(traj=traj,
                                         input_crs="EPSG:4326",
                                         output_crs="EPSG:3857",
                                         longitude="longitude",
                                         latitude="latitude")

traj['x'] = projected_x
traj['y'] = projected_y

traj.head(10)

## Apply desired filters

Our library currently allows for three common forms of filtering:
- **Temporal Filtering**: Restrict data to a time window of interest (e.g., January). Use the `start_time` and `end_time` arguments. If 
- **Quantity-Based Filtering**: Keep only users with sufficient activity as measured by a minimum number of pings. Use the `min_active_days` and `min_pings_per_day` arguments.
- **Spatial Filtering**: Keep only users with pings that fall within a specific geographic region (e.g., Philadelphia). Use the `polygon` argument.

If the aforementioned arguments are not specified, the default arguments ensure that the respective filtering is not performed. I.e., `polygon` defaults to `None`, and so no spatial filtering is performed.

In [ ]:
# Define the polygon for the park area (in projected coordinates)
polygon_coords = [
    (-4265504, 4393111), 
    (-4265504, 4393171), 
    (-4265564, 4393171), 
    (-4265564, 4393111)
]
polygon = Polygon(polygon_coords)

n0 = len(traj)
uq0 = traj['uid'].unique()

filtered_traj = filter_users(traj=traj, 
                             start_time=pd.Timestamp("2024-01-01 00:00:00", tz='America/New_York'),
                             end_time=pd.Timestamp("2024-01-07 23:59:00", tz='America/New_York'),
                             polygon=polygon,
                             min_active_days=2,
                             min_pings_per_day=10,
                             user_id='uid',
                             x='x',
                             y='y')

n1 = len(filtered_traj)
uq1 = filtered_traj['uid'].unique()
print(f"Number of pings before filtering: {n0}")
print(f"Number of unique users before filtering: {len(uq0)}")
print(f"Number of pings after filtering: {n1}")
print(f"Number of unique users after filtering: {len(uq1)}")


### Visualize completeness and filter by completeness metric

And here, a discussion on how we want to discard users that have too incomplete a signal to be used. E.g. Q < 80% . 

In [ ]:
#  _generate_Q_matrix(df, traj_cols={'datetime':'event_zoned_datetime', 'user_id':'cuebiq_id'})
# num_days = 42

# q_hourly = Q.mean(axis=0)*100
# q_daily = ((Q>0.0).sum(axis=0)/num_days)*100

# # assume Q is your DataFrame and q_daily is the Series of mean completeness per user
# sorted_cols = q_daily.sort_values(ascending=False).index
# Qs = Q[sorted_cols]
# binary = Qs.gt(0).astype(int)
# cmap = plt.cm.Blues

# fig, axes = plt.subplots(
#     2, 2,
#     figsize=(8, 8),
#     gridspec_kw={'height_ratios': [1, 2.5]},
#     sharey='row'
# )

# # marginal histograms
# axes[0,0].hist(q_daily, bins=46)
# axes[0,0].set_xlabel('% days with data')

# axes[0,1].hist(q_hourly, bins=46)
# axes[0,1].set_xlabel('% hours with data')

# for ax, data in zip([axes[0,1], axes[0,0]], (q_hourly, q_daily)):
#     ax.axvline(np.median(data), linestyle='--', color='red', lw=1)
# # binary heatmap
# sns.heatmap(
#     binary.T,
#     ax=axes[1,0],
#     cmap=cmap,
#     vmin=0, vmax=1,
#     cbar=False,
#     xticklabels=False,
#     yticklabels=False
# )
# axes[1,0].set_ylabel('User')

# # continuous heatmap
# sns.heatmap(
#     Qs.T,
#     ax=axes[1, 1],
#     cmap=cmap,
#     vmin=0, vmax=1,
#     cbar_kws={'label': 'Completeness'},
#     xticklabels=False,
#     yticklabels=False
# )
# axes[1,1].set_ylabel('')

# # set date ticks on bottom row
# dates = pd.to_datetime(Qs.index)
# tick_locs = np.linspace(0, len(dates) - 1, 6, dtype=int)
# tick_lbls = dates.strftime('%m-%d')[tick_locs]
# for ax in [axes[1,0], axes[1,1]]:
#     ax.set_xticks(tick_locs)
#     ax.set_xticklabels(tick_lbls, rotation=45, ha='right')

# #plt.tight_layout()
# plt.show()

The following code constructs a daily $Q$ matrix, which represents the completeness of user activity for each day. $q$ represents the proportion of hours in a day during which a specific user has recorded activity. A sliding window of 3 days is then applied to compute a mean completeness metric for each date in the sliding window.

In [ ]:
# Construct daily Q matrix
Q = filters._generate_Q_matrix(traj, tc)
Q

# no need for sliding window
# visualize (1) distribution of q=0, (2) barcode 0/1 for each hour for each user plt.imshow

In [ ]:
sliding_window = 3  # in days
SW_dates = Q.index[:-sliding_window]  # dates involved in the sliding window computation
Q_window = pd.DataFrame(
    [filters._compute_mean_q(Q, date, sliding_window) for date in SW_dates],
    index=SW_dates
    )

Q_window.head(5)

We now visualize the number of complete users in each day for different values of completeness ($\bar{q} = 1-\epsilon$).

In [ ]:
LIST_eps = [0.3,0.5]
Colors_eps = ['red', 'blue']
X = range(len(SW_dates))

DICT_legend= {'classes': [int(e*100) for e in LIST_eps], 
              'colors': Colors_eps, 
              'loc': 'lower right', 
              'fontsize':10,
              'title': '$\\epsilon \\quad (\\%)$', 
              'title_fontsize':10}

DICT_ticks = {'t': X, 
              'tl': SW_dates, 
              'rot': 90, 
              'size':10}

DICT_label_titles = {'xlabel': 'Date', 
                     'ylabel': 'Count of complete users', 
                     'title': f'Daily time-series of complete users \n over {sliding_window} days sliding window with $\\epsilon$ tolerance', 
                     'label_size': 15, 
                     'title_size': 15}

fig, ax = plt.subplots(figsize = (15,5))

for eps,color in zip(LIST_eps, Colors_eps):
    #Counts of complete users for each date - based on eps tolerance
    I_complete_eps = (Q_window > (1-eps)).sum(axis=1)
    ax.plot(X, I_complete_eps.values, color = color, label = eps)

filters._ax_visual_legend(ax, DICT_legend)
filters._ax_visual_ticklabel(ax, DICT_ticks, axis = 'x')
filters._ax_visual_labeltitles(ax, DICT_label_titles)

We can use $q$ to filter users by completeness, retaining only the users who exhibit a mean completeness of over `qbar` in the sample.

In [ ]:
n0 = len(traj)
uq0 = traj['uid'].unique()

complete_users = q_filter(traj=traj,
                          qbar=0.7,
                          sliding_window=3,
                          traj_cols=tc)
filtered_traj_q = traj[traj['uid'].isin(complete_users)]

n1 = len(filtered_traj_q)
uq1 = filtered_traj_q['uid'].unique()

print(f"Number of pings before filtering: {n0}")
print(f"Number of unique users before filtering: {len(uq0)}")
print(f"Number of pings after filtering: {n1}")
print(f"Number of unique users after filtering: {len(uq1)}")

## Applying filters on read

If your dataset is too large. Loading it all in memory to then apply the filters can be intractable. Nomad implements these functionalities to filter in this way on read...